In [13]:
6

6

In [14]:
# import FunctionalCollections
import Iterators
import Pipe
import Compat
import JLD
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
@everywhere using Compat
@everywhere using JLD

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [15]:
@everywhere fold_indexes=load("brown_glove_folds.jld","fold_indexes")

@everywhere function fold_split(fold_ii, raw_bow_res)
    ground_sents = Vector{ASCIIString}[rset[1] for rset in raw_bow_res]
    reconstructed_bows = Vector{ASCIIString}[rset[2] for rset in raw_bow_res]
    
    test_indexes = fold_indexes[fold_ii]
    training_indexes = trues(ground_sents)
    training_indexes[test_indexes]=false

    test_unordered_sents = reconstructed_bows[test_indexes]
    test_ground = ground_sents[test_indexes]
    training_sents = ground_sents[training_indexes]
    test_unordered_sents,test_ground, training_sents
end
    

In [16]:
@everywhere typealias S ASCIIString
@everywhere typealias State{T} Tuple{T,T}

import PyCall
@everywhere using PyCall
#http://www.nltk.org/howto/probability.html
@everywhere @pyimport nltk
@everywhere @pyimport nltk.probability as nltk_prob
@everywhere const START_MARKER1 = "**START1**"
@everywhere const START_MARKER2 = "**START2**"
@everywhere const END_MARKER1 = "**END1**"
@everywhere const END_MARKER2 = "**END2**"

@everywhere function train_language_model{T}(train_corpus::Vector{Vector{T}}; loglikelyhood::Bool=false)
    function py_collect(xs::PyObject)
        xst = []
        for x in xs
            push!(xst,x)
        end
        xst
    end
    function trigram_buffer(sent)
        [START_MARKER1, START_MARKER2, sent..., END_MARKER1, END_MARKER2] 
    end

    training_trigrams = vcat([py_collect(nltk.trigrams(trigram_buffer(sent))) for sent in train_corpus]...)
    kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, training_trigrams))
    
    function trigram_model(given1::S, given2::S, event::S)
        prob = kn_prob_dist[:prob]((given1, given2, event))
        loglikelyhood ? log(prob) : prob
    end
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "


In [17]:
@pyimport nltk.corpus as nltk_corpus
corpus_reader=nltk_corpus.brown
corpus = Vector{ASCIIString}[[lowercase(word) for word in sent] for sent in (corpus_reader[:sents]()|> collect)]
const log_lm = train_language_model(corpus, loglikelyhood=true)

trigram_model (generic function with 1 method)

In [18]:
using JuMP


"""
returns the a vector of sets of node indexes, each set is a subtour
The First subtour returned is the nonconnected one -- the path
"""
function get_subtours(x::Matrix{JuMP.Variable})
    x_val = getValue(x)
    x_iis,x_jjs, _  = findnz(x_val .>= 1 - 1e-6) #It just has to be close to 1 to be true
    nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])

    subtours = IntSet[]
    ii = START_NODE_INDEX
    push!(subtours, IntSet(END_NODE_INDEX)) #The END Node is always in the same subtour as the start node
    while(true)
        while(true) #Cycle through current subtour
            
            push!(subtours[end],ii) 
            
            jj = nodes_chain[ii]
            println(ii," ", jj)
            delete!(nodes_chain,ii)
            if jj∈subtours[end] 
                break 
            end
            ii=jj
        end   

        if length(nodes_chain)>0
            ii = first(keys(nodes_chain)) #start new subtour
            push!(subtours,IntSet())
        else
            break
        end
    end
    subtours
end

"""
Returns the tour from Start to End. ignores any subtours
"""
function get_coretour(x_links)
    nodes_chain = Dict([ii=>jj for (ii,jj) in x_links])

    tour = IntSet(END_NODE_INDEX)
    ii = START_NODE_INDEX
    while(ii∉tour) #Cycle through current subtour
        push!(tour,ii) 
        ii = nodes_chain[ii]
    end
    tour
end

function get_hyperclass(subtour)
    tour_hyper_class_nodes=IntSet()
    for ii in subtour
        w1,w2 = nodes[ii]
        #println(unordered_markers[w1])
        class_nodes = node_indexes_for_1st[w1]
        union!(tour_hyper_class_nodes,class_nodes) 
    end
    tour_hyper_class_nodes
end


get_hyperclass (generic function with 1 method)

In [19]:
corpus[1028]

20-element Array{ASCIIString,1}:
 "with"       
 "one"        
 "down"       
 "in"         
 "the"        
 "eighth"     
 ","          
 "marv"       
 "throneberry"
 "drew"       
 "a"          
 "walk"       
 "and"        
 "stole"      
 "second"     
 "as"         
 "hyde"       
 "fanned"     
 "tuttle"     
 "."          

In [65]:
corpus[120]


53-element Array{ASCIIString,1}:
 "it"         
 "would"      
 "authorize"  
 "the"        
 "texas"      
 "education"  
 "agency"     
 "to"         
 "establish"  
 "county-wide"
 "day"        
 "schools"    
 "for"        
 ⋮            
 "ones"       
 "to"         
 "attend"     
 "the"        
 "residential"
 "texas"      
 "school"     
 "for"        
 "the"        
 "deaf"       
 "here"       
 "."          

In [66]:
join((corpus[120])," ")

"it would authorize the texas education agency to establish county-wide day schools for the deaf in counties of 300,000 or more population , require deaf children between 6 and 13 years of age to attend the day schools , permitting older ones to attend the residential texas school for the deaf here ."

In [70]:
test_bag = shuffle(corpus[120]) #$ length 53
#test_bag = shuffle(corpus[21]) #$ length 33
#test_bag = shuffle(corpus[1022]) #length 28
#test_bag = shuffle(corpus[2000]) #length 24
#test_bag = shuffle(corpus[1028]) #length 20  (Gurodi 9.6, GLTK : 2029.6 seconds)
#test_bag = shuffle(corpus[1122]) #length 19 
#test_bag = shuffle(corpus[1000]) #length 17 (Gurbodi: 3.14 seconds GLTK:  569.9seconds)
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a","fine","comedy", "."])
#^length 9, Gurodi 0.62 seconds , GLTK 5.3seconds
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy", "."])
#test_bag =  shuffle(["it", "is", "so", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "good", "."])
#test_bag =  ["no", "way", "."]
#test_bag =  shuffle(["no", "."])
#@time best_order(test_bag, lm, mem_limit=1000)

53-element Array{ASCIIString,1}:
 "deaf"      
 "the"       
 "day"       
 "day"       
 ","         
 "here"      
 "would"     
 "of"        
 "13"        
 "texas"     
 "and"       
 "agency"    
 "permitting"
 ⋮           
 "6"         
 "."         
 "school"    
 "attend"    
 "to"        
 "years"     
 "the"       
 "it"        
 "for"       
 "of"        
 "population"
 "to"        

In [71]:
using JuMP
using MathProgBase
using GLPKMathProgInterface
using Gurobi

tic()
m=Model(solver=GurobiSolver(Threads=1))
#m=Model(solver=GurobiSolver(Threads=0))
#m=Model(solver=GLPKSolverMIP())



unordered_words  = test_bag
unordered_markers = [START_MARKER1; START_MARKER2; END_MARKER1; END_MARKER2; unordered_words...]
#Note that this lacks END_MARKER2
const START_NODE_INDEX = 1
const END_NODE_INDEX = 2

nodes = State{Int}[] #Named by word index

node_indexes_for_1st = Dict{Int, Vector{Int}}()
node_indexes_for_2nd = Dict{Int, Vector{Int}}()

function add_node!(ii,jj)
    push!(nodes, (ii,jj))

    node_indexes_for_i_1st = get!(()->Int[], node_indexes_for_1st, ii)
    push!(node_indexes_for_i_1st, length(nodes))

    node_indexes_for_j_2nd = get!(()->Int[], node_indexes_for_2nd, jj) 
    push!(node_indexes_for_j_2nd, length(nodes))
    #println("node:$(length(nodes)) |  $(unordered_markers[ii])($ii), $(unordered_markers[jj])($jj)")
end

add_node!(1, 2) #That is START_MARKER1-> START_MARKER2
add_node!(3, 4) #That is END_MARKER1-> END_MARKER2

for ii in 1:length(unordered_markers)
    wi = unordered_markers[ii]
    if wi∈(END_MARKER1,START_MARKER1) continue end  #Covered these
        
    for jj in [1:length(unordered_markers)]
        if ii==jj continue end
        wj = unordered_markers[jj]
        if wj∉(START_MARKER1,START_MARKER2,END_MARKER2)
            #but wj can be  END_MARKER1
            add_node!(ii,jj)
        end
    end
end

@defVar(m, x[1:length(nodes), 1:length(nodes)], Bin)

#If you enter a node you must also leave it
for (cc, center_node) in enumerate(nodes)
    if cc ∉ (START_NODE_INDEX, END_NODE_INDEX) #the beginning and end done have this requiement
        #Everything that enters this node, must leave this node
        @addConstraint(m, sum{x[ii,cc], ii=1:length(nodes)} == sum{x[cc,jj], jj=1:length(nodes)})
    end
end

rules = Dict{ASCIIString,Set{State{Int}}}()
for class_index in 1:length(unordered_markers)
    w_class =  unordered_markers[class_index]
    if w_class∉(START_MARKER1,START_MARKER2)
        #Not rquired to make a transition so that START_MARKER1 or 2 ever occur in second position
        jjs = node_indexes_for_2nd[class_index]
        @addConstraint(m, sum{x[ii,jj],ii=1:length(nodes), jj=jjs}==1)
        rules["*"*w_class]=Set(product(1:length(nodes),jjs))
    end
    
    #The following constraint is not required, as if it shows uo in the second position other rules make it certain to show up in the first
    #if w_class∉(END_MARKER1,END_MARKER2)
    #    #Not rquired to make a transition so that END_MARKER1 or 2 ever occur in second position
    #    iis=node_indexes_for_1st[class_index]
    #    @addConstraint(m, sum{x[ii,jj], ii=iis,jj=1:length(nodes)}==1)
    #    rules[w_class*"*"]=Set(product(iis,1:length(nodes)))
    #end
end

banned_trans_prob = Set{State{Int}}()
banned_trans_state = Set{State{Int}}()
log_trans_prob = spzeros(length(nodes), length(nodes))
for (from_node_index, from_node) in enumerate(nodes)
    w1 = unordered_markers[from_node[1]]
    w2 = unordered_markers[from_node[2]]
    #If what was in the second state element does not end up in the first state element then it is not allowed.
    can_transition_to = Set(get(node_indexes_for_1st, from_node[2], Int[]))
        #You can transition to any node which has your second element as its first element
    for (to_node_index, to_node) in enumerate(nodes)
        if to_node_index in can_transition_to
            @assert(from_node[2]==to_node[1])
            w3= unordered_markers[to_node[2]]
            log_tp = log_lm(w1,w2,w3)
            if log_tp>-Inf
                log_trans_prob[from_node_index, to_node_index] = log_tp
                continue
            else
                #Banned as prob zero transitions not allowed
                @addConstraint(m, x[from_node_index,to_node_index]==0)
                push!(banned_trans_prob, (from_node_index,to_node_index))
            end
            
        else
            #It is not a legal transition
            @addConstraint(m, x[from_node_index,to_node_index]==0)
            push!(banned_trans_state, (from_node_index,to_node_index))
        end
    end
end



function eleminate_subtours(cb)
    x_val = getValue(x)
    x_iis,x_jjs, _  = findnz(x_val .>= 1 - 1e-6) #It just has to be close to 1 to be true
    x_links = zip(x_iis,x_jjs)
    
    core_tour = get_coretour(x_links)
    if length(core_tour)==length(x_links)+1 #There is one more element in the core tour, as links vs fenceposts
        #no subtour to eleminate
        return
    end
    core_tour_hyperclasses = get_hyperclass(core_tour)
    
    #Also the core tour doesn't cover everything so there must be a link from one of its classes covered outside
    arcs_outof_coretour = AffExpr()
    for ii in core_tour_hyperclasses
        for jj in 1:length(nodes)
            if jj∉core_tour_hyperclasses
                arcs_outof_coretour += x[ii,jj]
            end
        end
    end
    @addLazyConstraint(cb, arcs_outof_coretour >=1)

end

addLazyCallback(m, eleminate_subtours,fractional=false)

#new Linear way using logprobs
@setObjective(m, Max, sum{log_trans_prob[i,j]*x[i,j], i=1:length(nodes), j=1:length(nodes)})

toc()
@printval length(unordered_words)
@printval length(m.linconstr)
num_banned_trans  = length(banned_trans_prob) + length(banned_trans_state)
variables = prod(size(log_trans_prob))
@printval variables
@printval variables-num_banned_trans
log_trans_prob


elapsed time: 323

2919x2919 sparse matrix with 29155 Float64 entries:
	[164 ,    2]  =  -0.0870114
	[323 ,    2]  =  -0.0281709
	[376 ,    2]  =  -1.38629
	[588 ,    2]  =  -0.133531
	[641 ,    2]  =  -1.38629
	[694 ,    2]  =  -1.38629
	[800 ,    2]  =  -0.133531
	[853 ,    2]  =  -0.0281709
	[906 ,    2]  =  -1.38629
	[959 ,    2]  =  -1.38629
	⋮
	[2177, 2919]  =  -10.1667
	[2230, 2919]  =  -10.1669
	[2283, 2919]  =  -10.1668
	[2336, 2919]  =  -10.1669
	[2442, 2919]  =  -10.1667
	[2495, 2919]  =  -10.1668
	[2601, 2919]  =  -10.5937
	[2707, 2919]  =  -10.76
	[2760, 2919]  =  -10.1669
	[2813, 2919]  =  -10.1669
	[2866, 2919]  =  -10.5724

In [ ]:
tic()
solve(m)
time_to_solve = toc()
println("Objective value: ", getObjectiveValue(m))
println("Prob: ", e^getObjectiveValue(m))
x_val = getValue(x)
#println("x = ", x_val)
x_iis,x_jjs, _  = findnz(x_val.>1-1e-6)
zip(x_iis,x_jjs) |> collect |> println

In [ ]:
(time_to_solve + 49.9)

In [ ]:
nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])
node_index=1
visitted = Set{Int}()
while(node_index!=END_NODE_INDEX)
    push!(visitted, node_index)
    println(unordered_markers[nodes[node_index][1]])
    #println(node_index, " ", unordered_markers[nodes[node_index][1]]," ", unordered_markers[nodes[node_index][2]])
    node_index = nodes_chain[node_index]
end

In [ ]:
x_val = getValue(x)
x_iis,x_jjs, _  = findnz(x_val)
nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])

In [ ]:
x

In [ ]:
collect(1:100)[[1]]

In [ ]:
sts = get_subtours(x)

In [ ]:
1+1

In [ ]:
for tran in union(banned_trans_prob, banned_trans_state)
    for rule in values(rules)
        delete!(rule, tran)
    end
end
rules

In [ ]:
rules["fine*"] ∩ keys(nodes_chain)

In [ ]:
function get_prod(x, iis, jjs)
    net = 1.0
    for i in 1:size(x,1) 
        for j in 1:size(x,2)
            println(i,",", j, " =", x[i,j], " ", trans_prob[i,j])
            net*=max((x[i,j]-1)^2, trans_prob[i,j])
        end
    end
    net
end
values = falses(size(x))
values[1,3] = 1
values[3,6] = 1
values[6,7] = 1
println("----------\n")
get_prod(values, iis,jjs)
